In [20]:

#pip install pytesseract
#pip install PyPDF2
#pip install pdfplumber
#pip install ocrmypdf

In [ ]:
#para trabajar con ocrmypdf se debe intalar adicionalmente: Ghostscript y Tesseract OCR


In [32]:
import os
import pytesseract
from PIL import Image
import PyPDF2
import ocrmypdf